In [ ]:
# %pip install scikit-learn
# %pip install pandas

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import pandas as pd
import time, concurrent.futures, json, requests, re

In [ ]:
cosine_similarity([[3,1]],[[6,2]])
cosine_distances([[3,1]],[[6,2]])

In [ ]:
cosine_similarity([[3,0]],[[0,8]])

In [ ]:
cosine_similarity([[3,1]],[[3,2]])

In [ ]:
doc1 = """
iphone sales contributed to 70% of revenue. iphone demand is increasing by 20% yoy. 
the main competitor phone galaxy recorded 5% less growth compared to iphone" ipad ipad 
"""

doc2 = """
The upside pressure on volumes for the iPhone 12 series, historical outperformance 
in the July-September time period heading into launch event, and further catalysts in relation
to outperformance for iPhone 13 volumes relative to lowered investor expectations implies a 
very attractive set up for the shares.
"""

doc3 = """
samsung's flagship product galaxy is able to penetrate more into asian markets compared to
iphone. galaxy is redesigned with new look that appeals young demographics. 60% of samsung revenues
are coming from galaxy phone sales
"""

doc4 = """
Samsung Electronics unveils its Galaxy S21 flagship, with modest spec improvements 
and a significantly lower price point. Galaxy S21 price is lower by ~20% (much like the iPhone 12A), 
which highlights Samsung's focus on boosting shipments and regaining market share.
"""

In [ ]:
df = pd.DataFrame([
        {'iPhone': 3,'galaxy': 1, 'ipad': 4, 'hi': 0},
        {'iPhone': 2,'galaxy': 0, 'ipad': 1, 'hi': 0},
        {'iPhone': 1,'galaxy': 3, 'ipad': 0, 'hi': 0},
        {'iPhone': 1,'galaxy': 2, 'ipad': 2, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
        {'iPhone': 0,'galaxy': 0, 'ipad': 0, 'hi': 0},
    ],
    index=[
        "doc1",
        "doc2",
        "doc3",
        "doc4",
        "doc5","doc6","doc7","doc8"
    ])

In [ ]:
df
#df.loc["doc1":"doc1"]

In [ ]:
cosine_similarity(df.loc["doc1":"doc1"],df.loc["doc5":"doc5"])

In [ ]:
data =  ["independent","abruptness","abandonment"]
response_API = requests.post("http://127.0.0.1:5000/searchdata", json = {"word": "carvet"})

sendData = response_API.text
print(sendData)

In [ ]:
# request to get books
def getBooksThread(bookId, timeout=10):
    response_API = requests.get('https://gutendex.com/books/{}'.format(bookId), timeout=timeout)
    data = response_API.text
    parse_json = json.loads(data)
    if parse_json.get('detail') != None:
        # print(bookId)
        return 'NOT_FOUND'
    return parse_json 

def getBooksData(listBooks):
    print("Running get books threads:")
    threaded_start = time.time()
    booksData = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for bookId in listBooks:
            futures.append(executor.submit(getBooksThread, bookId))
        for future in concurrent.futures.as_completed(futures):
            if future.result() != 'NOT_FOUND':
                booksData.append(future.result())
            # print(future.result())
    print("Threaded get books", time.time() - threaded_start)
    return booksData



def getListBooks(listBooks):
    def transformData(d):
        res = []
        if d.get('formats')!=None:
            for t in d['formats'].keys():
                checkEnd = d['formats'][t].split('.').pop()
                if checkEnd == 'txt':
                    res.append({
                        'id': d['id'],
                        'text_url': d['formats'][t]
                    })
        return res
    
    data = getBooksData(listBooks)
    result = []
    
    print("Running get list books:")
    threaded_start = time.time()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for d in data:
            futures.append(executor.submit(transformData, d))
        for future in concurrent.futures.as_completed(futures):
            result += future.result()
    print("Threaded get list books time:", time.time() - threaded_start)
    return result

def getTableIndex(listBooks):
    tableIndex = dict()
    booksInfo = []
    listBooksData = getListBooks(listBooks)

    for book in listBooksData:
        response_API = requests.get(book['text_url'])
        data = response_API.text
        #### Option 1: Prendre seulement des mots avec carateres de 4 à 10
        words = re.findall(r"[A-Za-z]{4,10}\w+", data)
        occurentCounts = dict()

        for word in words:
            w = word.lower()

            # Count for table index all books
            if w in tableIndex:
                if book['id'] in tableIndex[w]:
                    tableIndex[w][book['id']] += 1
                else:
                    tableIndex[w][book['id']] = 1
            else:
                tableIndex[w] = dict({book['id']: 1})

            # Count for table index for each book
            if w in occurentCounts:
                occurentCounts[w] += 1
            else:
                occurentCounts[w] = 1

        booksInfo.append({
            "bookId": book['id'],
            "words": occurentCounts,
            "totalWords": len(words),
            "totalWordsWithOccur": len(occurentCounts)
        })    
    return tableIndex, booksInfo



In [ ]:

listBooks = [49345,56667,1,2,3,4,5,6,7]
tableIndexData, booksInfo = getTableIndex(listBooks)
historyWords =dict({
  "carver": 1, 
  "carvet": 1, 
  "saigon": 2, 
  "sargon": 3
})

print(tableIndexData['sargon'])


In [12]:

def cosineSimilarity(historyWords, tableIndexData):
    # Init variable
    result = dict()
    if historyWords!=None:
        booksData = dict({'history':historyWords})
        for word in historyWords.keys():
            if word in tableIndexData:
                for b in tableIndexData[word].keys(): 
                    if b in booksData:
                        booksData[b].update(dict({word:tableIndexData[word][b]}))
                    else:
                        booksData.update(dict({b: dict({word:tableIndexData[word][b]})}))
        bookDF = pd.DataFrame(booksData.values(),
            index=booksData.keys()).fillna(0)
        for cs in list(booksData.keys())[1:]:
            result[cs] = cosine_similarity(bookDF.loc["history":"history"],bookDF.loc[cs:cs])[0][0]
        return result
    else:
        booksData = dict()

        for word in tableIndexData:
            for b in tableIndexData[word].keys(): 
                if b in booksData:
                    booksData[b].update(dict({word:tableIndexData[word][b]}))
                else:
                    booksData.update(dict({b: dict({word:tableIndexData[word][b]})}))
        bookDF = pd.DataFrame(booksData.values(),
            index=booksData.keys()).fillna(0)
        print(bookDF)
        pass

res = cosineSimilarity(None, tableIndexData)
print(res)


       welcome  world  plain  vanilla  electronic  texts  etexts  readable  \
4          1.0      3      3        1           1      1    13.0         3   
1          4.0     22      6        2           2      3    17.0         7   
5          1.0      3      3        1           1      2     3.0         4   
2          1.0      2      3        1           1      1    13.0         3   
6          1.0      3      3        1           1      1    13.0         3   
3          3.0     10      3        1           1      1    13.0         3   
49345      2.0     26     47        2          27     99     0.0         2   
7          1.0      2      3        1           1      1    13.0         3   
56667      0.0     39     22        2          27     55     0.0         2   

       humans  computers  ...  profound  griechische  klassische  altertum  \
4         1.0          2  ...       0.0          0.0         0.0       0.0   
1         2.0          3  ...       0.0          0.0         0.

In [ ]:
booksData = []
for book in booksInfo:
    count = 0
    for w in historyWords.keys():
        if w in book['words']:
            # count number occurs in document
            # count += book['words'][w]
            # count if have that word in ducument
            count += historyWords[w]
    booksData.append({
        "bookId": book['bookId'],
        "jaccard": (count/book['totalWordsWithOccur'])
    })

[print(b) for b in booksData]